In [62]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce

# 读取数据集

本次数据分为 train.csv 和 test.csv。每个文件有10列，前9列为特征 （都为离散型），最后一列是标签（±1）。

In [31]:
def loadDataSet(filePath):
    ''' 数据集读取函数'''
    data, label = [], []
    # 读取数据集
    with open(filePath) as f:
        for line in f.readlines():
            temp = line.strip().split(",")
            data.append([float(i) for i in temp[:-1]])
            if temp[-1] != '?':
                temp[-1] = float(temp[-1])
            label.append(temp[-1])
    ##### 输出数据集相关信息 ##########
    print("data dimension of dataset：", len(data[0]))
    print("number of sample in data :", len(data))
    print("label frequency:", dict(Counter(label)))
    ##### 输出数据集相关信息 ##########
    return np.array(data), np.array(label)

In [82]:
trainSet, trainSet_label = loadDataSet('.\\data\\train.csv')
trainSet[0:5]

data dimension of dataset： 9
number of sample in data : 787
label frequency: {1.0: 323, -1.0: 464}


array([[ 23.,   1.,   2.,   2.,   1.,   1.,   2.,   0.,   0.],
       [ 36.,   1.,   1.,   5.,   1.,   1.,   2.,   3.,   0.],
       [ 33.,   1.,   3.,   3.,   1.,   1.,   2.,   3.,   0.],
       [ 35.,   3.,   3.,   2.,   1.,   0.,   0.,   3.,   0.],
       [ 25.,   1.,   2.,   4.,   1.,   1.,   2.,   2.,   0.]])

In [81]:
testSet, testSet_lable = loadDataSet('.\\data\\test.csv')
testSet[0:5]

data dimension of dataset： 9
number of sample in data : 300
label frequency: {'?': 300}


array([[ 32.,   3.,   3.,   3.,   0.,   1.,   0.,   3.,   0.],
       [ 24.,   1.,   2.,   3.,   1.,   1.,   1.,   2.,   0.],
       [ 32.,   3.,   3.,   2.,   1.,   0.,   0.,   2.,   0.],
       [ 43.,   3.,   3.,   5.,   1.,   1.,   0.,   3.,   0.],
       [ 46.,   3.,   3.,   1.,   0.,   1.,   0.,   3.,   0.]])

# 特征选取

## 信息增益和信息增益率

In [99]:
def calcInfoGain_or_InfoGainRate(dataSet, label, calcInfoGainRate=False):
    '''计算数据集每一列（特征）的信息增益或信息增益率'''
    def calcEntropy(data):
        '''计算单列数据的熵'''
        probs_ = np.array(list(Counter(data).values()))/data.shape[0]
        ans = -1*(probs_*np.log2(probs_)).sum()
        return ans
    #计算数据集的熵
    dataSetEntropy = calcEntropy(label)
    #得到数据集的 样本数 和 特征数
    sampleNum, featureNum = dataSet.shape
    infoGains = np.zeros(featureNum) #用于保存 信息增益的数组
    #对于数据集的每一个特征
    for featureId in range(featureNum):
        #得到当前的 特征
        curFeature = dataSet[:, featureId]
        #得到每个取值的统计次数
        counter = Counter(curFeature)
        #得到所有可能的取值
        values = list(counter.keys())
        #得到所有可能取值的概率
        probs = np.array(list(counter.values()))/sampleNum
        entropys = np.zeros(len(values)) #用于保存熵的数组
        #遍历每个可能的取值
        for index, val in enumerate(values):
            #得到 标签 中对应的 子数据集标签
            subLabel = label[np.argwhere(curFeature==val)[:,0]]
            #计算 子数据集标签 的 熵
            entropys[index] = calcEntropy(subLabel)
        #计算基于当前特征的 条件熵
        condEntropy = (probs*entropys).sum()
        #计算基于当前特征的 信息增益
        infoGains[featureId] = dataSetEntropy - condEntropy
        #若是计算 信息增益率，则要除以 当前特征的 熵 
        if calcInfoGainRate:
            infoGains[featureId] /= calcEntropy(curFeature)
    return infoGains

In [95]:
ansA = calcInfoGain_or_InfoGainRate(trainSet, trainSet_label, False)
ansA
np.argmax(ansA)

array([ 0.08227668,  0.01368315,  0.0152763 ,  0.10408631,  0.00169982,
        0.00107348,  0.00518738,  0.00597883,  0.00918736])

3

In [94]:
ansB = calcInfoGain_or_InfoGainRate(trainSet, trainSet_label, True)
ansB
np.argmax(ansB)

array([ 0.01673974,  0.00701169,  0.00964848,  0.03349103,  0.00286384,
        0.00129432,  0.00321961,  0.00328779,  0.01890093])

3

## 基尼指数

In [103]:
def calcGiniIndex(dataSet, label):
    '''计算数据集每一列（特征）的Gini指数'''
    #得到数据集的 样本数 和 特征数
    sampleNum, featureNum = dataSet.shape
    giniIndexs = np.zeros(featureNum) #用于保存 信息增益的数组
    #对于数据集的每一个特征
    for featureId in range(featureNum):
        #得到当前的 特征
        curFeature = dataSet[:, featureId]
        #得到每个取值的统计次数
        counter = Counter(curFeature)
        #得到所有可能的取值
        values = list(counter.keys())
        #得到所有可能取值的概率
        probs = np.array(list(counter.values()))/sampleNum
        subGiniIndexs = np.zeros(len(values)) #用于保存熵的数组
        #遍历每个可能的取值
        for index, val in enumerate(values):
            #得到 标签 中对应的 子数据集
            subLabel = label[np.argwhere(curFeature==val)[:,0]]
            #计算 每个取值下 数据集的 gini指数
            sub_values = np.array(list(Counter(subLabel).values()))
            sub_probs = sub_values / subLabel.shape[0]
            subGiniIndexs[index] = 1 - (sub_probs**2).sum()
        #计算基于当前特征的 gini 指数
        giniIndexs[featureId] = (probs*subGiniIndexs).sum()
    return giniIndexs

In [104]:
ansC = calcGiniIndex(trainSet, trainSet_label)
ansC
np.argmax(ansC)

array([ 0.43710387,  0.47500663,  0.47507976,  0.42602918,  0.48279887,
        0.48323448,  0.4804874 ,  0.48007143,  0.47810829])

5